<a href="https://colab.research.google.com/github/KruglikovAnton/megafons_clients_investigation/blob/main/megafon_investigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

![](https://sun9-32.userapi.com/C6vN5pHQW7hlseIt0AtTN7qNDQbDnHXdt1P2NA/ILntXqa-RrA.jpg)

## Codebook

`megafon.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `user_id` — идентификатор абонента;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Q1` — ответ на первый вопрос;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Q2` — ответ на второй вопрос;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Total Traffic(MB)` — объем трафика передачи данных <sup>1 </sup>; <br>
&nbsp;&nbsp;&nbsp;&nbsp; `Downlink Throughput(Kbps)` — средняя скорость «к абоненту» <sup>2 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Uplink Throughput(Kbps)`— средняя скорость «от абонента» <sup>3 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Downlink TCP Retransmission Rate(%)` — частота переотправок пакетов «к абоненту» <sup>4 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Video Streaming Download Throughput(Kbps)` — скорость загрузки потокового видео <sup>5 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Video Streaming xKB Start Delay(ms)` — задержка старта воспроизведения видео <sup>6 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Web Page Download Throughput(Kbps)` — скорость загрузки web-страниц через браузер <sup>7 </sup>;<br>
&nbsp;&nbsp;&nbsp;&nbsp; `Web Average TCP RTT(ms)` — пинг при просмотре web-страниц<sup>8 </sup>.<br>


<sup>1 </sup> — Насколько активно абонент использует мобильный интернет.<br>
<sup>2 </sup> — Считается по всему трафику передачи данных.<br>
<sup>3 </sup> — Считается по всему трафику передачи данных.<br>
<sup>4 </sup> — Чем выше, тем хуже. Если в канале возникает ошибка, пакет переотправляется. Снижается полезная скорость.<br>
<sup>5 </sup> — Чем выше, тем лучше — меньше прерываний и лучше качество картинки.<br>
<sup>6 </sup> — Сколько времени пройдёт между нажатием на кнопку Play и началом воспроизведения видео. Чем меньше это время, тем быстрее начинается воспроизведение.<br>
<sup>7 </sup> — Чем выше, тем лучше.<br>
<sup>8 </sup> — Чем меньше, тем лучше — быстрее загружаются web-страницы.<br>

Первый технический показатель представлен как сумма за период в одну неделю перед участием в опросе. Остальные технические показатели отображают среднее значение по данному признаку за период в одну неделю перед участием в опросе.

In [ ]:
start_df = pd.read_csv('megafon.csv')
start_df.head(5)

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
0,1,5,NaN,775.48846,360.13,86.56,3.93,1859.15,2309,1007.82,83
1,2,5,4,861.96324,3023.54,411.18,1.27,667.47,2080,255.36,425
2,3,1,4,261.11860,790.96,34.20,1.79,1079.60,6367,535.85,485
3,4,8,3,179.18564,2590.97,325.88,0.80,7053.81,3218,1221.02,51
4,5,2,"2, 3, 4",351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68


In [ ]:
start_df.describe(include='all')

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
count,3112.000000,3110,1315,3112.000000,3112.000000,3112.000000,3112.000000,3112.000000,3112.000000,3112.000000,3112.000000
unique,NaN,49,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,10,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,846,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1556.500000,NaN,NaN,406.282339,2220.774023,170.702381,1.834518,5728.806848,2107.596080,2100.258901,194.195051
std,898.501345,NaN,NaN,220.593815,2629.824405,204.031286,2.074593,5161.475235,1582.423057,2062.509549,267.387914
min,1.000000,NaN,NaN,100.026680,13.260000,6.110000,0.050000,0.000000,239.000000,0.000000,0.000000
25%,778.750000,NaN,NaN,214.261192,652.297500,70.735000,0.720000,2324.550000,1280.000000,1018.002500,74.000000
50%,1556.500000,NaN,NaN,363.999720,1404.230000,119.435000,1.300000,4436.500000,1672.000000,1670.695000,118.000000
75%,2334.250000,NaN,NaN,572.524325,2823.645000,210.617500,2.240000,7611.440000,2312.000000,2519.987500,207.000000


В Q1 и Q2 49 и 51 уникальных значения из общей выборки в 3110 и 1315 соответственно, что связано с своим вариантом ответа или nan.

In [ ]:
#Отсортируем по значению Q1
start_df.sort_values('Q1')

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
491,492,***** ** ***,NaN,479.50838,8675.19,230.06,0.33,6818.51,1562,2797.14,52
158,159,0,NaN,126.51761,361.56,46.89,0.82,965.21,3564,1709.08,90
1133,1134,0,NaN,359.66875,3578.95,58.31,2.34,5634.10,1066,1745.89,87
1991,1992,0,NaN,140.70227,3575.30,150.95,0.83,7929.53,1884,3048.05,35
117,118,0,NaN,837.55274,2511.24,113.10,0.39,4976.77,2244,1544.19,66
...,...,...,...,...,...,...,...,...,...,...,...
834,835,"Чем даль ше,тем лучше.Спасибо за ваш труд.Оцен...",NaN,886.05369,1115.41,937.26,1.20,4439.31,1930,753.67,109
3041,3042,"Я в Смол. Области живу сейчас, не пользуюсь те...",NaN,338.04205,167.67,57.20,1.16,363.19,782,166.81,637
1365,1366,Я ценой услуг не удовлетворен,NaN,543.14687,4818.84,81.10,0.72,6786.28,1129,2494.68,73
371,372,NaN,NaN,286.62191,2320.25,75.60,3.84,4499.92,1790,1035.64,200


Как видно из отсортированной таблицы существуют значения, где в Q1 и Q2 присутствуют nan, а также значения, в которых в Q1 0, а в Q2 nan. На данном этапе уберем подобные значения.

In [ ]:
main_df = start_df[start_df.Q1.isin(list(map(str, list(range(1,10)))))]
main_df.describe(include='all')

Как видим это уменьшило число данных на треть, что говорит о необходимости их исследования